## CIFAR-10을 활용한 이미지 분류 실습

### 데이터 설명


+ 데이터 셋 : **[cifar-10](https://www.cs.toronto.edu/~kriz/cifar.html)**(Canadian Institute For Advanced Research)
    + 10개의 class와 50,000개의 train data, 10,000개의 test data로 구성
    + 각 image는 32 x 32, 총 1,024개의 pixel로 이루어진 컬러 이미지
    
    + 각 pixel마다 RGB의 가중치 값(정수형, 0 ~ 255)이 [R, G, B] 형태로 들어 있음
    + 아래 그림은 10개의 class의 라벨과 그에 따른 대표 이미지
---
+ 변수 설명
    + x_train : Array 형태의 50,000개의 train 이미지
    + y_train : Array 형태의 50,000개의 train label : 10개의 class가 (0 ~ 9)의 정수로 인코딩
    
    + x_test : Array 형태의 10,000개의 test 이미지
    + y_test : Array 형태의 10,000개의 test label : 10개의 class가 (0 ~ 9)의 정수로 인코딩

In [ ]:
# load CIFAR-10 dataset
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
import numpy as np

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)
print('===========================')
# Data preprocessing (nomalized)
x_train = x_train.astype(np.float32) / 255.
x_test = x_test.astype(np.float32) / 255.

# Data preprocessing (one-hot encoding)
# number => 원래는 0~9로 클래스가 명시되어 있지만
#           실제 class에 해당하는 부분만 1로 채워지고
#           나머지는 0으로 채워진 one-hot encoding 형태의 클래스로 변경됨.
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

In [ ]:
import random
import matplotlib.pyplot as plt
class_names = { 0:'Airplane', 1:'Car', 2:'Bird', 3:'Cat', 4:'Deer',
               5:'Dog', 6:'Frog', 7:'Horse', 8:'Boat', 9:'Truck' }

def preview_data(X_train, y_train):
    plt.figure(figsize=(10, 8))
    for c in range(10):
        label = np.zeros((10,))
        label[c] = 1
        print(label)
        i = random.choice(np.where(y_train == label)[0])
        plt.subplot(5, 5, c+1)
        plt.axis('off')
        plt.title(f"{class_names[c]}: {c}")
        plt.imshow(X_train[i])

preview_data(x_train, y_train)

### 신경망 모델 설계

In [ ]:
# basic 합성곱신경망 설계
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout

model = Sequential()

##### ▽ 코드 작성 ▽ #####

# 2D conv이기 때문에 input은 3D
# 공간적인 특징을 분류하기 때문
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(.25))
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(10, activation='softmax'))


##### △ 코드 작성 △ #####

In [ ]:
# 모델 요약
model.summary()

### 신경망 모델 학습

In [ ]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'Adam',
    metrics=['accuracy']
)
# 손실함수 최소화해야 함.

In [ ]:
from tensorflow.python.ops.variables import validate_synchronization_aggregation_trainable
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 128, #2의 제곱으로 설정
    epochs = 10,
    validation_data = (x_test, y_test),
    verbose=2
)

### 학습 진행상황 시각화

In [ ]:
def draw_loss_plot(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='best')
    plt.grid()
    plt.show()

In [ ]:
draw_loss_plot(history)

In [ ]:
def draw_accuracy_plot(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='best')
    plt.grid()
    plt.show()

In [ ]:
draw_accuracy_plot(history)

### 학습 신경망 모델 저장

In [ ]:
model.save("my_cnn.h5")